# Imports

In [ ]:
import torch
from torch.optim import AdamW

%pip install wget

from transformers import SentimentTransformer
from utils import enwik8, train_model, test_model, create_sentence_batches, load_imdb

Note: you may need to restart the kernel to use updated packages.


# Get Train and Test Data

In [13]:
"""
x_train, x_val: List[List[int]] -> integer encoded sentences
y_train, y_val: List[int] -> labels for each sentence
i2w -> int to word mapping
w2i -> word to int mapping
numcls -> Number of labels
"""
(x_train, y_train), (x_val, y_val), (i2w, w2i), numcls = load_imdb(final=False)
path = r"C:\Users\Hp\Desktop\Coding\Thesis\enwik8 (1).gz"
wiki_data = enwik8(path)

In [30]:
wiki_data[2].size()

torch.Size([5000000])

In [ ]:
def sample_batch(data, seq_len, batch_size):
    """ Breaks down a sequence of chars into a batch of subsequences of chars.
    :return: batch of input sequences for the model and corresponding target sequences shifted by 1 char.
    """
    # randomly select batch_size no. of starting points for seqs in the batch
    start_pts = torch.randint(size=batch_size, low=0, high=data.size(0) - seq_len - 1)
    inputs, targets = [], []
    for st in start_pts:
        input = data[st : st + seq_len]
        target = data[st + 1 : st + seq_len + 1]        
        inputs.append(input[None, :])
        targets.append(target[None, :])

    inputs = torch.cat(inputs, dim=0).to(torch.long)
    targets = torch.cat(targets, dim=0).to(torch.long)


# Model Initialisation and Batch Creation

In [ ]:
# creating batches
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 32
train_batches = create_sentence_batches(x_train, y_train, batch_size=32, device=device)
val_batches = create_sentence_batches(x_val, y_val, batch_size=32, device=device)

# model init
embedding_dim = 300
vocab_size = len(w2i)
model = SentimentTransformer(embedding_dim, vocab_size, numcls, heads=1, num_encoders=1, mask=True).to(device)

# model training
epochs = 10
learning_rate = 0.001
optimizer = AdamW(model.parameters(), lr=learning_rate)
# train_model(train_batches, model, optimizer, learning_rate=learning_rate, epochs=epochs, print_progress=True)

# model testing
# test_model(val_batches, model, print_metrics=True)